In [1]:
from flyvis.datasets.moving_bar import MovingEdge
import numpy as np
import os
from flyvis_cell_type_pert import FlyvisCellTypePert
import pandas as pd
from tqdm import tqdm
from moving_edge import MovingEdgeWrapper
from pathlib import Path
from flyvis_cell_type_pert import PerturbationType


 -> Patched datamate.directory._write_h5
Importing flyvis...


In [2]:
import datamate

data_path = Path("data/flyvis_data")
data_path.mkdir(parents=True, exist_ok=True)

env = os.environ.copy()
env["FLYVIS_ROOT_DIR"] = str(data_path)


# def fixed_write_h5(path, val):
#     """
#     A Windows-safe replacement that skips the 'read-before-write' check.
#     """
#     path = Path(path)
#     path.parent.mkdir(parents=True, exist_ok=True)

#     # Simple, safe write. No checking 'f["data"]', no 'unlink'.
#     # This completely bypasses the logic causing your KeyError.
#     with h5py.File(path, mode="w", libver="latest") as f:
#         f.create_dataset("data", data=val)
# datamate.io._write_h5 = fixed_write_h5
# if hasattr(datamate.directory, "_write_h5"):
#     datamate.directory._write_h5 = fixed_write_h5
#     print(" -> Patched datamate.directory._write_h5")
# else:
#     print(" -> Warning: Could not find _write_h5 in directory module")


In [3]:
cell_type_df = pd.read_csv(f'{data_path}/flyvis_cell_type_connectivity.csv')

dataset = MovingEdge(
            offsets=[-10, 11],  # offset of bar from center in 1 * radians(2.25) led size
            intensities=[0, 1],  # intensity of bar
            speeds=[19],  # speed of bar in 1 * radians(5.8) / s
            height=80,  # height of moving bar in 1 * radians(2.25) led size
            post_pad_mode="continue",  # for post-stimulus period, continue with the last frame of the stimulus
            t_pre=1.0,  # duration of pre-stimulus period
            t_post=1.0,  # duration of post-stimulus period
            dt=1 / 200,  # temporal resolution of rendered video
            angles=list(np.arange(0, 360, 30)),  # motion direction (orthogonal to edge)
    )

In [5]:
split_dfs = np.array_split(cell_type_df, 3)

df_part1 = split_dfs[0]
df_part2 = split_dfs[1]
df_part3 = split_dfs[2]

c:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
# wrapper = MovingEdgeWrapper(dataset, pert=None, pert_folder_name=None,
#                                 output_file_name="data/flyvis_data/perf/original_network.csv")
# wrapper.run()

for src, tar in tqdm(df_part1[['source_type', 'target_type']].values):
    result_output_path = f'data/flyvis_data/perf/{src}_{tar}.csv'
    if os.path.exists(result_output_path):
        print(f"Skipping existing perturbation: {src} -> {tar}")
        continue
    print(f"Running perturbation: {src} -> {tar}")

    pert = FlyvisCellTypePert()
    pert.perturb(cell_type_df, PerturbationType.PAIR_WISE, pairs=[(src, tar)])

    wrapper = MovingEdgeWrapper(dataset, pert=pert,
                                pert_folder_name=f'{src}_{tar}_perturbation',
                                output_file_name=result_output_path)
    wrapper.run()

  0%|          | 0/202 [00:00<?, ?it/s][2026-01-06 23:07:29] network_view:122 Initialized network view at C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_Am_C2_perturbation


Running perturbation: Am -> C2


[2026-01-06 23:07:32] network:222 Initialized network with NumberOfParams(free=734, fixed=2959) parameters.
[2026-01-06 23:07:32] chkpt_utils:36 Recovered network state.


Running moving edge simulation...
Applying perturbation to network in memory...
Overwriting disk checkpoints with perturbed weights...


[2026-01-06 23:07:33] network_view:122 Initialized network view at C:\Users\dean\Documents\dev\fly_winter_school\fly_wire_perturbations\data\flyvis_data\results\flow\0000\000_Am_C2_perturbation


 -> Updated: data\flyvis_data\results\flow\0000\000_Am_C2_perturbation\best_chkpt
 -> Updated: data\flyvis_data\results\flow\0000\000_Am_C2_perturbation\chkpts\chkpt_00000
Clearing caches...
 -> Removed __cache__
Generating moving edge responses...
